In [ ]:
% matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#pretty plot stuff
#I use this to make changes to my plot
plt.rcParams['figure.figsize'] = (8,6.5)           # changes my figure size with length 8 and width 6.5
plt.rcParams['legend.frameon'] = True              # if True, legend will have a border
plt.rcParams['legend.fontsize'] = 12               # legend fontsize is 12
plt.rcParams['legend.borderpad'] = 0.5             # width of the border of the legend
plt.rcParams["legend.framealpha"] = 0.8            # changes how transparant the legend is.
plt.rcParams['font.family'] = 'STIXGeneral'        # sets font
plt.rcParams['font.size'] = 20                     #sets general font size

# Integrate the function f(x) = e^-2x * cos(10x)
Use the trapezoid, Simpson's method, and Romberg integration.
Integrate over the range [0,pi]

In [ ]:
x = np.linspace(0, np.pi, 1000)

#define the actual function we want to get the integral of
def func(x):
    a = -2
    b = 10
    return np.exp(a*x) * np.cos(b*x)

#integral solution from Wolfram Alpha
def func_integral(x):
    a = -2
    b = 10
    return (-1/52) * np.exp(a*x) * (np.cos(b*x) - 5*np.sin(b*x))

plt.plot(x, func(x))
plt.plot(x, func_integral(x))
plt.axhline(0, color = 'black', ls = '--', lw = 0.8)
plt.grid()

# Trapezoid Method

In [ ]:
#define the core of the trapezoid method
def trapezoid_core(f, x, h):
    return 0.5 * h * (f(x+h) + f(x)) #area of the trapezoid where h is like dx and f(x+h) is y2 and f(x) is y1

#define a wrapper function to perform the trapezoid method
def trapezoid_method(f, a, b, N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evalutions to use
    
    #define x values to perform trapezoid rule
    x = np.linspace(a, b, N)
    h = x[1] - x[0]
    
    #define the value of the ingtegral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0, len(x) - 1, 1):
        Fint += trapezoid_core(f, x[i], h)
        
    #return the answer
    return Fint


# Simpson's Method

In [ ]:
#define the core of the Simpson's method
def simpsons_core(f, x, h):
    return h * (f(x) + 4*f(x + h) + f(x + 2*h))/3.

#define a wrapper function to perform Simpson's Method
def simpsons_method(f, a, b, N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of function evalutions to use
    # Note: the number of chunks will be N - 1, so if N is odd don't adjust
    
    #define x values to perform trapezoid rule
    x = np.linspace(a, b, N)
    h = x[1] - x[0]
    
    #define the value of the ingtegral
    Fint = 0.0
    
    #if N is odd or even, we have different numbers of chunks to add
    if((N % 2) == 0):
        lim = 3
    else:
        lim = 2
    
    #perform the integral using the trapezoid method
    for i in range(0, len(x) - 2, 2):
        Fint += simpsons_core(f, x[i], h)
        #print(i, i + 2, x[i] + h, x[i] + 2*h, x[-2], x[-1])

    #apply simpson's rule over the last interval when N is even
    if (N % 2 == 0):
        Fint += simpsons_core(f, x[-2], 0.5*h)
        
    #return the answer
    return Fint

# Romberg Integration

In [ ]:
#define the Romberg integration core
def romberg_core(f, a, b, i):
    #we need the difference b-a
    h = b - a
    
    #and the incremement between the new function evalutations
    dh = h/2.**(i)
    
    #we need the cofactor
    K = h/2.**(i + 1)
    
    #and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
    
    #return the answer
    return K*M

#define a wrapper function to perform Romberg Integration
def romberg_integration(f, a, b, tol):
    #define an iteration variable 
    i = 0
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error estimate, and set to a large value
    delta = 100.0 * np.fabs(tol)
    
    #set an array of interal answers
    I = np.zeros(imax, dtype = float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5 * (b - a) * (f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta > tol):
        #find this romberg iteration
        I[i] = 0.5*I[i - 1] + romberg_core(f, a, b, i)
        
        #compute the new functional error estimate
        delta = np.fabs( (I[i] - I[i - 1])/I[i])
        
        print(i, I[i], I[i - 1], delta)
        
        if(delta > tol):
            #iterate
            i += 1
            
            #if we reach the maximum iterations...
            if(i > imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after ', i)
                
    #return the answer
    return I[i]

# Integral Check

In [ ]:
Answer = func_integral(1) - func_integral(0)
print(Answer)

N = 10
tolerance = 1.0e-4 #larger tolerance for trapezoid method because it is too slow
#I will use a while loop to find the best possible N to find the asnwer
print("Trapezoid Method")
trap_answ = trapezoid_method(func, 0 , 1, N)

while np.abs(trap_answ - Answer)/Answer > tolerance:
    trap_answ = trapezoid_method(func, 0 , 1, N)
    #print(trapezoid_method(func, 0 , 1, N))
    N+=1
print(trap_answ, ", Intervals: " + str(N), ", Tolerance: " + str(tolerance))

N = 10
tolerance = 1.0e-6
#I will use a while loop to find the best possible N to find the asnwer
print("Simpson's Method")
simp_answ = simpsons_method(func, 0 , 1, N)
while np.abs(simp_answ - Answer)/Answer > tolerance:
    simp_answ = simpsons_method(func, 0 , 1, N)
    N += 1
print(simp_answ, ", Intervals: " + str(N), ", Tolerance: " + str(tolerance))

print("Romberg Integration")
tolerance = 1.0e-6
RI = romberg_integration(func, 0 , 1, tolerance)
print(RI, ", Error: " + str(np.abs(RI - Answer)/Answer), ", Tolerance: " + str(tolerance))